In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import joblib
from rdkit import Chem
from rdkit.Chem import AllChem 
import os
from torch_geometric.data import Data, DataLoader
import pandas as pd
import numpy as np
import numpy as np # linear algebra
import pandas as pd 
import xgboost as xgb
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Performing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.inspection import permutation_importance
import shap
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib
import numbers
import torch
from torch_geometric.loader import DataLoader  

In [ ]:
# 直接加载当前目录下的 style.mplstyle
plt.style.use('style.mplstyle')
import matplotlib.pyplot as plt

# 打印当前使用的字体
print("font", plt.rcParams["font.family"])

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, dropout_rate=0.3):
        super(MLP, self).__init__()
        layers = []
        in_dim = input_size

        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(in_dim, hidden_size))        
            layers.append(nn.BatchNorm1d(hidden_size))                
            layers.append(nn.ReLU())                                 
            layers.append(nn.Dropout(dropout_rate))                 
            in_dim = hidden_size                                   
        
        layers.append(nn.Linear(in_dim, output_size))
        
        self.network = nn.Sequential(*layers)

    def forward(self, data):
        x = data.morgan_fp
        out = self.network(x) 
        return out

In [ ]:
import torch

def load_data_list(file_path):
    return torch.load(file_path)

############## Tm ##############
Tm_load_file_path = '../dataset/Tm_data_list.pt' 
Tm_load_train_dataset_path = '../dataset/train_Tm_data_list.pt' 
Tm_load_val_dataset_path = '../dataset/val_Tm_data_list.pt' 
Tm_load_test_dataset_path = '../dataset/test_Tm_data_list.pt' 


Tm_loaded_data_list = load_data_list(Tm_load_file_path)
Tm_loaded_train_data_list = load_data_list(Tm_load_train_dataset_path)
Tm_loaded_val_data_list = load_data_list(Tm_load_val_dataset_path)
Tm_loaded_test_data_list = load_data_list(Tm_load_test_dataset_path)

Tm_train_loader = DataLoader(Tm_loaded_train_data_list, batch_size=32, shuffle=True)  
Tm_val_loader = DataLoader(Tm_loaded_val_data_list, batch_size=32, shuffle=False)  
Tm_test_loader = DataLoader(Tm_loaded_test_data_list, batch_size=32, shuffle=False) 


############## conductivity ##############
conductivity_load_file_path = 'conductivity_data_list.pt'
conductivity_load_train_dataset_path = 'train_conductivity_data_list.pt' 
conductivity_load_val_dataset_path = 'val_conductivity_data_list.pt'
conductivity_load_test_dataset_path = 'test_conductivity_data_list.pt' 

conductivity_loaded_data_list = load_data_list(conductivity_load_file_path)
conductivity_loaded_train_data_list = load_data_list(conductivity_load_train_dataset_path)
conductivity_loaded_val_data_list = load_data_list(conductivity_load_val_dataset_path)
conductivity_loaded_test_data_list = load_data_list(conductivity_load_test_dataset_path)


conductivity_train_loader = DataLoader(conductivity_loaded_train_data_list, batch_size=32, shuffle=True)  
conductivity_val_loader = DataLoader(conductivity_loaded_val_data_list, batch_size=32, shuffle=False)  
conductivity_test_loader = DataLoader(conductivity_loaded_test_data_list, batch_size=32, shuffle=False)  

######################ECW######################
IL_ECW_save_total_dataset_path = '../dataset/IL_ECW_data_list.pt' 
IL_ECW_save_train_dataset_path = '../dataset/train_IL_ECW_data_list.pt' 
IL_ECW_save_val_dataset_path = '../dataset/val_IL_ECW_data_list.pt'
IL_ECW_save_test_dataset_path = '../dataset/test_IL_ECW_data_list.pt'

IL_ECW_loaded_data_list = load_data_list(IL_ECW_save_total_dataset_path)
IL_ECW_loaded_train_data_list = load_data_list(IL_ECW_save_train_dataset_path)
IL_ECW_loaded_val_data_list = load_data_list(IL_ECW_save_val_dataset_path)
IL_ECW_loaded_test_data_list = load_data_list(IL_ECW_save_test_dataset_path)

IL_ECW_train_loader = DataLoader(IL_ECW_loaded_train_data_list, batch_size=32, shuffle=True)  
IL_ECW_val_loader = DataLoader(IL_ECW_loaded_val_data_list, batch_size=32, shuffle=False)  
IL_ECW_test_loader = DataLoader(IL_ECW_loaded_test_data_list, batch_size=32, shuffle=False)  

In [ ]:
def testing_mlp(model, test_loader,scale=1, input_type="fp"):
    model.to(device)
    model.eval()

    total_loss = 0
    test_pred = []
    test_true = []
    loss_fn = torch.nn.MSELoss() 

    # 在测试集上进行评估
    for data in test_loader:
        data = data.to(device)
        if input_type=="fp":
            data.morgan_fp = data.morgan_fp.float()
        elif input_type=="2Ddescriptors":
            data.moldescriptor = data.moldescriptor.float()
        out = model(data)
        loss = loss_fn(out, data.y.view(-1, 1))  
        total_loss += loss.item()
        test_pred.append(out.detach().cpu().numpy())
        test_true.append(scale*data.y.cpu().numpy())

    test_pred = np.concatenate(test_pred, axis=0)
    test_true = np.concatenate(test_true, axis=0)
    average_loss = total_loss / len(test_loader)

    test_pred_flat = test_pred.flatten() 
    test_true_flat = test_true.flatten()
    test_mse = mean_squared_error(test_true_flat, test_pred_flat)
    test_mae = mean_absolute_error(test_true_flat, test_pred_flat)
    test_r2 = r2_score(test_true_flat, test_pred_flat)
    return test_pred, test_true, test_mse, test_mae, test_r2

In [ ]:
def testing_GNN(model, test_loader,scale=1):
    model.to(device)
    model.eval()

    total_loss = 0
    test_pred = []
    test_true = []
    loss_fn = torch.nn.MSELoss() 

    for data in test_loader:
        data = data.to(device)
        data.x = data.x.float()
        out = model(data)
        loss = loss_fn(out, data.y.view(-1, 1)) 
        total_loss += loss.item()
        test_pred.append(out.detach().cpu().numpy())
        test_true.append(scale*data.y.cpu().numpy())

    test_pred = np.concatenate(test_pred, axis=0)
    test_true = np.concatenate(test_true, axis=0)
    average_loss = total_loss / len(test_loader)

    test_pred_flat = test_pred.flatten() 
    test_true_flat = test_true.flatten() 
    test_mse = mean_squared_error(test_true_flat, test_pred_flat)
    test_mae = mean_absolute_error(test_true_flat, test_pred_flat)
    test_r2 = r2_score(test_true_flat, test_pred_flat)

    print(f"Test Loss: {average_loss:.4f}, MSE: {test_mse:.4f}, MAE: {test_mae:.4f}, R2: {test_r2:.4f}")
    
    return test_pred, test_true, test_mse, test_mae, test_r2 

In [ ]:
def plot_targets(pred, ground_truth, mse, mae, save_path="GAT_val.png", figsize=(2.3, 2.3), 
                 marker_style="o", scale_factor=1):

    plt.rcParams["font.family"] = "Arial"
    plt.rcParams["font.size"] = 7

    pred_scaled = [p * scale_factor for p in pred]
    ground_truth_scaled = [g * scale_factor for g in ground_truth]

    r2 = r2_score(ground_truth, pred)

    fig, ax = plt.subplots(figsize=figsize)
    
    ax.scatter(pred_scaled, ground_truth_scaled, s=10, marker=marker_style, facecolors='none', edgecolors='grey')
    
    ax.axline((1, 1), slope=1, color="red", linestyle="--")
    
    ax.set_xlabel("Predicted Value")
    ax.set_ylabel("Ground Truth")
    
    text_str = f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.2f}"
    ax.text(0.05, 0.95, text_str,
            transform=ax.transAxes,
            fontsize=7, color="black",
            verticalalignment="top", horizontalalignment="left")

    for spine in ax.spines.values():
        spine.set_linewidth(0.75)

    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def extract_features_targets(data_list, scale=1, feature = "fp"):

    X = []
    Y = []
    for data in data_list:
        if feature == "2Ddescriptors":
            moldescriptor = data.moldescriptor.numpy().flatten()
            X.append(moldescriptor)
            
        elif feature == "fp":
            fp = data.morgan_fp.numpy().flatten()
            X.append(fp)
            
        target = data.y.numpy().flatten()
        Y.append(scale*target)
    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

In [ ]:
def evaluate_model(model, val_list, scale=1, figname="model_evaluation",
                   figsize=(2.3, 2.3), output=False, feature="fp", marker_style="o"):

    X_test, y_test = extract_features_targets(val_list, scale, feature)
    

    plt.rcParams["font.family"] = "Arial"
    plt.rcParams["font.size"] = 7
    
    y_pred = model.predict(X_test)
    print(y_pred)
    
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"MSE (Mean Squared Error): {mse:.2f}")
    print(f"MAE (Mean Absolute Error): {mae:.2f}")
    print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
    print(f"R^2 Score: {r2:.2f}")
    
    fig, ax = plt.subplots(figsize=figsize)
    ax.scatter(y_pred, y_test, s=10, marker=marker_style, facecolors='none', edgecolors='grey')
    
    ax.axline((1, 1), slope=1, color="red", linestyle="--")
    
    ax.set_xlabel("Predicted Value")
    ax.set_ylabel("Ground Truth")

    text_str = f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.2f}"
    ax.text(0.05, 0.95, text_str,
            transform=ax.transAxes,
            fontsize=7, color="black",
            verticalalignment="top", horizontalalignment="left",
            bbox=dict(facecolor='white', alpha=0.5, edgecolor='none'))

    for spine in ax.spines.values():
        spine.set_linewidth(0.75)
    
    if output:
        plt.savefig(f"{figname}.png", dpi=300, bbox_inches="tight")
    
    plt.show()
    return mae, mse, r2

In [ ]:
import os  
import sys 
sys.path.insert(0, "/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties") 

from GATModel import GAT
from GCNModel import GCN
from MLPModel import MLP

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# GAT
model_Tm_GAT = GAT(in_channels=5,hidden_channels=256).to(device)  
model_conductivity_GAT = GAT(in_channels=5,hidden_channels=256).to(device)  
model_IL_ECW_GAT = GAT(in_channels=5,hidden_channels=256).to(device) 
# GCN
model_Tm_GCN = GCN(hidden_dim=128).to(device)
model_conductivity_GCN = GCN(hidden_dim=128).to(device) 
model_IL_ECW_GCN = GCN(hidden_dim=128).to(device) 
# MLP
input_size = 2048
hidden_sizes = [256, 64]
output_size = 1
model_Tm_MLP = MLP(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size).to(device) 
model_conductivity_MLP = MLP(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size).to(device) 
model_IL_ECW_MLP = MLP(input_size=input_size, hidden_sizes=hidden_sizes, output_size=output_size).to(device) 

In [ ]:
# conductivity model
MLP_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/conductivity_MLP_model.pt'
GCN_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/conductivity_GCN_model.pt'
GAT_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/conductivity_GAT_model.pt'

MLP_state_dict = torch.load(MLP_model_path, map_location=device)
GCN_state_dict = torch.load(GCN_model_path, map_location=device)
GAT_state_dict = torch.load(GAT_model_path, map_location=device)

model_conductivity_MLP.load_state_dict(MLP_state_dict)
model_conductivity_GCN.load_state_dict(GCN_state_dict)
model_conductivity_GAT.load_state_dict(GAT_state_dict)

conductivity_xgb_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/conductivity_xgb_model.joblib'
conductivity_xgb_model = joblib.load(conductivity_xgb_model_path)

# Tm model
MLP_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/Tm_MLP_model.pt'
GCN_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/Tm_GCN_model.pt'
GAT_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/Tm_GAT_model.pt'

MLP_state_dict = torch.load(MLP_model_path, map_location=device)
GCN_state_dict = torch.load(GCN_model_path, map_location=device)
GAT_state_dict = torch.load(GAT_model_path, map_location=device)

model_Tm_MLP.load_state_dict(MLP_state_dict)
model_Tm_GCN.load_state_dict(GCN_state_dict)
model_Tm_GAT.load_state_dict(GAT_state_dict)

Tm_xgb_model_path = '/data/jupyternotebook_public/Project15_AI_ML_BMS_JZZ/cemp/ionic_liquid/static/model/Tm_xgb_model_fp.joblib'
Tm_xgb_model = joblib.load(Tm_xgb_model_path)

# ECW model
MLP_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/IL_ECW_MLP_model.pt'
GCN_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/IL_ECW_GCN_model.pt'
GAT_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/IL_ECW_GAT_model.pt'

MLP_state_dict = torch.load(MLP_model_path, map_location=device)
GCN_state_dict = torch.load(GCN_model_path, map_location=device)
GAT_state_dict = torch.load(GAT_model_path, map_location=device)

model_IL_ECW_MLP.load_state_dict(MLP_state_dict)
model_IL_ECW_GCN.load_state_dict(GCN_state_dict)
model_IL_ECW_GAT.load_state_dict(GAT_state_dict)

IL_ECW_xgb_model_path = '/data/jupyternotebook_public/jifengwang/17_HTQC_analysis/AI_prediction_properties/predict_Tm_conductivity/IL_ECW_xgb_model.joblib'
IL_ECW_xgb_model = joblib.load(IL_ECW_xgb_model_path)


In [ ]:
# GCN evaluation
Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, Tm_test_r2_mlp = testing_GNN(model_Tm_GCN , Tm_test_loader)
plot_targets(Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, save_path="Tm_GCN_test.png"
             , marker_style="o")

IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, IL_ECW_test_r2 = testing_GNN(model_IL_ECW_GCN , IL_ECW_test_loader)
plot_targets(IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, save_path="IL_ECW_GCN_test.png"
             , marker_style="^")

conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, conductivity_test_r2 = testing_GNN(model_conductivity_GCN , conductivity_test_loader, scale=10)
plot_targets(conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, save_path="conductivity_GCN_test.png"
             , marker_style="s")

In [ ]:
# GAT evaluation
Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, Tm_test_r2_mlp = testing_GNN(model_Tm_GAT , Tm_test_loader)
plot_targets(Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, save_path="Tm_GAT_test.png"
             , marker_style="o")

IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, IL_ECW_test_r2 = testing_GNN(model_IL_ECW_GAT , IL_ECW_test_loader)
plot_targets(IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, save_path="IL_ECW_GAT_test.png"
             , marker_style="^")

conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, conductivity_test_r2 = testing_GNN(model_conductivity_GAT , conductivity_test_loader)
plot_targets(conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, save_path="conductivity_GAT_test.png"
             , marker_style="s")

In [ ]:
# MLP evaluation
Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, Tm_test_r2_mlp = testing_mlp(model_Tm_MLP , Tm_test_loader)
plot_targets(Tm_test_pred, Tm_test_true, Tm_test_mse, Tm_test_mae, save_path="Tm_MLP_test.png"
             , marker_style="o")

IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, IL_ECW_test_r2 = testing_mlp(model_IL_ECW_MLP , IL_ECW_test_loader)
plot_targets(IL_ECW_test_pred, IL_ECW_test_true, IL_ECW_test_mse, IL_ECW_test_mae, save_path="IL_ECW_MLP_test.png"
             , marker_style="^")

conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, conductivity_test_r2 = testing_mlp(model_conductivity_MLP , conductivity_test_loader, scale=10)
plot_targets(conductivity_test_pred, conductivity_test_true, conductivity_test_mse, conductivity_test_mae, save_path="conductivity_MLP_test.png"
             , marker_style="s")

In [ ]:
# XGBoost evaluation
evaluate_model(Tm_xgb_model, Tm_loaded_test_data_list, scale=1, figname="Tm_xgboost_model_evaluation",
                   figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="o")

evaluate_model(IL_ECW_xgb_model, IL_ECW_loaded_test_data_list, scale=1, figname="IL_ECW_xgboost_model_evaluation",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="^")
evaluate_model(conductivity_xgb_model, conductivity_loaded_test_data_list, scale=10, figname="conductivity_xgboost_model_evaluation",
               figsize=(2.3, 2.3), output=True, feature = "fp", marker_style="s")